In [1]:
%%capture
!pip install gradio && pip install diffusers && pip install transformers && pip install accelerate && pip install git+https://github.com/facebookresearch/segment-anything.git

In [2]:
import gradio as gr
import numpy as np
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
class CFG:
    device = 'cuda'
    sam_checkpoint = '/kaggle/input/segment-anything-models/sam_vit_h_4b8939.pth'
    model_type = 'vit_h'

In [4]:

sam = sam_model_registry[CFG.model_type](checkpoint = CFG.sam_checkpoint)
sam.to(CFG.device)
predictor = SamPredictor(sam)

In [5]:
pipe = StableDiffusionInpaintPipeline.from_pretrained('stabilityai/stable-diffusion-2-inpainting', torch_dtype = torch.float16)
pipe = pipe.to(CFG.device)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/clip/feature_extraction_clip.py:31: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  FutureWarning,


In [6]:
selected_pixels = []

what do we do? build a demo, we input an image, select a part, replace the part give na prompt - or replace the background instead of the object

In [7]:
with gr.Blocks() as demo:
    with gr.Row():
        input_img = gr.Image(label = "Input")
        mask_img = gr.Image(label = "Mask")
        output_img = gr.Image(label = "Output")
                
    with gr.Row():
        prompt_text = gr.Textbox(lines = 1, label = "Prompt")
        
    with gr.Row():
        submit = gr.Button("Submit")

    def generate_mask(image, evt: gr.SelectData ):
        
        selected_pixels.append(evt.index)

        predictor.set_image(image)
        input_points= np.array(selected_pixels)
        input_labels = np.ones(input_points.shape[0]) 
        mask, _, _ = predictor.predict(
          point_coords = input_points,
          point_labels = input_labels,
          multimask_output = False
      )

        # for background change
        mask = np.logical_not(mask)
        mask = Image.fromarray(mask[0,:,:])
        return mask

    def inpaint(image, mask, prompt):
        image = Image.fromarray(image)
        mask = Image.fromarray(mask)
        image = image.resize((512,512))
        mask = mask.resize((512,512))

        output = pipe(
            prompt = prompt, 
            image = image, 
            mask_image = mask).images[0]

        return output
    
    input_img.select(generate_mask, [input_img], [mask_img])
    submit.click(inpaint, inputs = [input_img, mask_img, prompt_text],
                 outputs = [output_img])
    



In [ ]:
demo.launch(debug = True)

Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://2b54cd4bc173cd6814.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


  0%|          | 0/50 [00:00<?, ?it/s]